In [1]:
import openpyxl
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import range_boundaries, get_column_letter
#from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import PatternFill
#https://stackoverflow.com/questions/72746988/merge-cells-using-python
from openpyxl import load_workbook

In [2]:
import xlwings as xw
#https://docs.xlwings.org/en/stable/
#https://www.geeksforgeeks.org/working-with-excel-files-in-python-using-xlwings/

In [3]:

from copy import copy
import os
import re

In [78]:
adresa_tabs = r"c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m"
filenames = [f"{adresa_tabs}\\{file}" for file in os.listdir(adresa_tabs) if file.lower().endswith(".xlsx")]
files = [file for file in os.listdir(adresa_tabs) if file.lower().endswith(".xlsx")]
files.sort()
print(files)
#filenames = sorted(filenames)

#for index, filename in enumerate(sorted(filenames)):
#    print(filename)

['brand_m_1.XLSX', 'brand_m_10.XLSX', 'brand_m_11.XLSX', 'brand_m_2.XLSX', 'brand_m_3.XLSX', 'brand_m_4.XLSX', 'brand_m_5.XLSX', 'brand_m_6.XLSX', 'brand_m_7.XLSX', 'brand_m_8.XLSX', 'brand_m_9.XLSX', 'formated_.xlsx']


None


## Bere syrové tabulky přímo z spss.
   - načte tabulku, překopíruje do nové tabulky bez spss naformátování, aby se pak dali spustit format a signifikance
   - zkusíme rovnou spustet format a signifikance

In [76]:

counter = 0
    
adresa_tabs = r"c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m"
filenames = (f"{adresa_tabs}\\{file}" for file in os.listdir(adresa_tabs) if file.lower().endswith(".xlsx"))
files = [f"{file}" for file in os.listdir(adresa_tabs) if file.lower().endswith(".xlsx")]
# Tuple of filenames
#filenames = ("OUTPUT_2.xlsx",
#             "OUTPUT.xlsx",
#            )




# column_num is the next column number to be written to in the new spreadsheet
column_num = 1
header = False

# Create a new workbook
new_wb = openpyxl.Workbook() 

# Read each workbook, and copy each column, cell by cell, to the new spreadsheet
for index, filename in enumerate(filenames):
    #print(filename)
    wb = openpyxl.load_workbook(filename)
    ws = wb.active
    
    # Create a new workbook
    #new_wb = openpyxl.Workbook()
    #new_ws = new_wb.active
    #https://stackoverflow.com/questions/26328398/how-to-write-two-sheets-in-a-single-workbook-at-the-same-time-using-openpyxl
    new_ws = new_wb.create_sheet()
    new_ws.title = files[index]
    
    counter += 1
        
    # The min and max columns of every MergedCellRange needs to be shifted
    print(filename)
    #print(min_col_2,":", min_row_2,":", max_col_2,":", max_row_2)
    for range_ in ws.merged_cells.ranges:
        #print(range_)
        min_col, min_row, max_col, max_row = range_boundaries(range_.coord)
        min_col = get_column_letter(min_col)
        max_col = get_column_letter(max_col)
        min_row = min_row
        max_row = max_row
        #print(min_col,":", min_row,":", max_col,":", max_row)
        new_ws.merged_cells.add(f"{min_col}{min_row}:{max_col}{max_row}")
        


    for row in ws.iter_rows():
        for cell in row:
            #rounding
            #print(cell.value)
            if type(cell.value) == float:
                cell.value = round(cell.value, 3)
            new_cell = new_ws.cell(row=cell.row, column=cell.column, value=cell.value)
                #print(cell.value)

    #Hiding gridlines     
    new_ws.sheet_view.showGridLines = False
    #smazani automaticky generovaného sheetu:
    if counter == 1:
        del new_wb['Sheet']
    # Save the new workbook to disk
    new_filename = f"{adresa_tabs}\\formated\\formated_.xlsx"
    #print(dir(new_wb.save())
    new_wb.save(new_filename)
    
    # Launch the new spreadsheet
    #os.startfile(new_filename)

    #break

    








c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_1.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_10.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_11.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_2.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_3.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_4.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_5.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_6.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_7.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_8.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_9.XLSX


## Sloučení zformátovaných tabulek

In [153]:
counter = 0
counter_find_width = 0
    

# Tuple of filenames
filenames = ("OUTPUT_formated_2.xlsx",
             "OUTPUT_formated.xlsx",
            )

# Create a new workbook
new_wb = openpyxl.Workbook()
new_ws = new_wb.active

# column_num is the next column number to be written to in the new spreadsheet
column_num = 1
row_num = 1
header = False
width_list = []

def find_coord_formated_tabs (ws):
    counter = 0
    for row in ws.iter_rows():
        counter += 1
        for cell in row:
            if cell.fill.start_color.index == 22:
                return counter
                
                #print (cell.fill.start_color.index)
                

for filename in filenames:
    #print(filename)
    wb = openpyxl.load_workbook(filename)
    ws = wb.active

    wb_xlwing = xw.Book(filename)
    ws_xlwing=wb_xlwing.sheets.active

    counter_find_width += 1

    for col in range(ws.min_column, ws.max_column + 1):
        width_to_append = ws_xlwing.range(str(get_column_letter(col)+"1")).column_width
        if counter_find_width == 1:
            width_list.append(width_to_append)
        else:
            if width_list[col - 1] < width_to_append:
                width_list[col - 1] = width_to_append
                
    print(len(width_list))
    print(width_list)
    #print(ws.max_column + 1)
    wb_xlwing.close()

# Read each workbook, and copy each column, cell by cell, to the new spreadsheet
for filename in filenames:
    #print(filename)
    wb = openpyxl.load_workbook(filename)
    ws = wb.active
    
    counter += 1
    if counter == 1:
        print(filename)
    max_row_2 = find_coord_formated_tabs (ws)
    print(max_row_2)

    # The min and max columns of every MergedCellRange needs to be shifted
    if counter == 1:
        print(filename)
        #print(min_col_2,":", min_row_2,":", max_col_2,":", max_row_2)
        for range_ in ws.merged_cells.ranges:
            #print(range_)
            min_col, min_row, max_col, max_row = range_boundaries(range_.coord)
            min_col = get_column_letter(min_col)
            max_col = get_column_letter(max_col)
            min_row = min_row + row_num
            max_row = max_row + row_num
            #print(min_col,":", min_row,":", max_col,":", max_row)
            new_ws.merged_cells.add(f"{min_col}{min_row-1}:{max_col}{max_row-1}")
        count_row = 0
    elif counter > 1:
        print(filename)
        for range_ in ws.merged_cells.ranges:
            min_col, min_row, max_col, max_row = range_boundaries(range_.coord)
            min_col = get_column_letter(min_col)
            max_col = get_column_letter(max_col)
            min_row = min_row + row_num
            max_row_check = max_row
            max_row = max_row + row_num
            if max_row_check <= max_row_2:
                print(max_row_2)
                print(range_ )
                continue
            #print(min_col,":", min_row,":", max_col,":", max_row)
            new_ws.merged_cells.add(f"{min_col}{min_row-(max_row_2+1)}:{max_col}{max_row-(max_row_2+1)}")
        count_row = max_row_2+1
        
            
    count_for_width = 0
    for row in ws.iter_rows(min_row=count_row):
        count_for_width += 1
        for cell in row:
            #print(cell)
            #rounding
            if type(cell.value) == float:
                cell.value = round(cell.value, 3)
                #print(cell.value)
            new_cell = new_ws.cell(row=row_num, column=cell.column, value=cell.value)
            
            #D    
            #print(cell.fill)
            #red_fill = PatternFill(start_color="ff0000", fill_type = "solid")
            #print (cell.fill.start_color.index)
            #if cell.fill.start_color.index == "FFE0E0E0":
                #new_cell.fill = red_fill
                #print (cell.fill.start_color.index)
             


            # Styles have to be manually copied
            if cell.has_style:
                new_cell.font = copy(cell.font)
                new_cell.border = copy(cell.border)
                new_cell.fill = copy(cell.fill)
                new_cell.number_format = copy(cell.number_format)
                new_cell.protection = copy(cell.protection)
                new_cell.alignment = copy(cell.alignment)
                
            # Preserve the same column width                
                #print(cell)
                #print(width)
            #if count_for_width == 1:
                #if type(cell).__name__ != 'MergedCell':
                    #width = ws.column_dimensions[cell.column_letter].width
                    #new_column = new_ws.column_dimensions[new_cell.column_letter]
                    #print(cell)
                    #print(width)
                    #print(new_column.width)
                    #new_column.width = width
                    #print("zmena na: ", new_column.width)
            
        row_num += 1
    
    ####vložení šířky z původních tabulek:
    if counter == 1:
        dim_holder = DimensionHolder(worksheet=ws)
        for col in range(ws.min_column, ws.max_column + 1):            
            dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=width_list[col - 1])
            #print(dim_holder[get_column_letter(col)])

        new_ws.column_dimensions = dim_holder
        
    ###Zkopírování ukotvení příčky:
    if counter == 1:
        new_ws.freeze_panes = ws.freeze_panes

    #break
    
#Hiding gridlines
new_ws.sheet_view.showGridLines = False


# Save the new workbook to disk
new_filename = "combined.xlsx"
new_wb.save(new_filename)

# Launch the new spreadsheet
os.startfile(new_filename)

91
[30.0, 30.0, 15.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0]
91
[30.0, 30.0, 15.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12

## Zkopírování šířky sloupců přes openpyxl. nenachází mi to sloupec B a nevím proc, proto nepoužito

In [ ]:
    ####vložení šířky z původních tabulek:
# dim_holder = DimensionHolder(worksheet=ws)
#    for col in range(ws.min_column, ws.max_column + 1):            
#         width = ws.column_dimensions[get_column_letter(col)].width
#         new_width = new_ws.column_dimensions[get_column_letter(col)].width
#         #print(width)
#         #print(new_width)
#         #print(ws.column_dimensions[get_column_letter(col)].width)
#         if width > new_width:
#             width_to_apply = width
#         else:
#             width_to_apply = new_width
#         dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=width_to_apply)
#         #print(dim_holder[get_column_letter(col)])
#    new_ws.column_dimensions = dim_holder

## Color handling: jen jako vzor

In [ ]:
            #print(cell.fill)
            red_fill = PatternFill(start_color="ff0000", fill_type = "solid")
            new_cell = new_ws.cell(row=row_num, column=cell.column, value=cell.value)
            #print (cell.fill.start_color.index)
            if cell.fill.start_color.index == "FFE0E0E0":
                new_cell.fill = red_fill
                #print (cell.fill.start_color.index)

## Spojuje syrové tabulky přímo z spss. musí se vyjet jako xlsx

In [152]:


counter = 0
    

# Tuple of filenames
filenames = ("OUTPUT_2.xlsx",
             "OUTPUT.xlsx",
            )


# Create a new workbook
new_wb = openpyxl.Workbook()
new_ws = new_wb.active

# column_num is the next column number to be written to in the new spreadsheet
column_num = 1
row_num = 1
header = False

def find_coord_raw_tabs (ws):
    for range_ in ws.merged_cells.ranges:
        #print(range_)
        if re.match(r"^A1:", str(range_)) and counter == 1:
            min_col_2, min_row_2, max_col_2, max_row_2 = range_boundaries(range_.coord)
            break
    return min_col_2, min_row_2, max_col_2, max_row_2
    

# Read each workbook, and copy each column, cell by cell, to the new spreadsheet
for filename in filenames:
    #print(filename)
    wb = openpyxl.load_workbook(filename)
    ws = wb.active
    
    counter += 1
    if counter == 1:
        #print(filename)
        min_col_2, min_row_2, max_col_2, max_row_2 = find_coord_raw_tabs (ws)
        
    # The min and max columns of every MergedCellRange needs to be shifted
    if counter == 1:
        print(filename)
        #print(min_col_2,":", min_row_2,":", max_col_2,":", max_row_2)
        for range_ in ws.merged_cells.ranges:
            #print(range_)
            min_col, min_row, max_col, max_row = range_boundaries(range_.coord)
            min_col = get_column_letter(min_col)
            max_col = get_column_letter(max_col)
            min_row = min_row + row_num
            max_row = max_row + row_num
            #print(min_col,":", min_row,":", max_col,":", max_row)
            new_ws.merged_cells.add(f"{min_col}{min_row-1}:{max_col}{max_row-1}")
        count_row = 0
    elif counter > 1:
        print(filename)
        for range_ in ws.merged_cells.ranges:
            min_col, min_row, max_col, max_row = range_boundaries(range_.coord)
            min_col = get_column_letter(min_col)
            max_col = get_column_letter(max_col)
            min_row = min_row + row_num
            max_row_check = max_row
            max_row = max_row + row_num
            if max_row_check <= max_row_2:
                print(max_row_2)
                print(range_ )
                continue
            #print(min_col,":", min_row,":", max_col,":", max_row)
            new_ws.merged_cells.add(f"{min_col}{min_row-(max_row_2+1)}:{max_col}{max_row-(max_row_2+1)}")
        count_row = max_row_2+1
        


    for row in ws.iter_rows(min_row=count_row):
        for cell in row:
            #rounding
            if type(cell.value) == float:
                cell.value = round(cell.value, 3)
                #print(cell.value)
            new_cell = new_ws.cell(row=row_num, column=cell.column, value=cell.value)
                
            
        row_num += 1
             


            # Styles have to be manually copied
            #if cell.has_style:
            #    new_cell.font = copy(cell.font)
            #    new_cell.border = copy(cell.border)
            #    new_cell.fill = copy(cell.fill)
            #    new_cell.number_format = copy(cell.number_format)
            #    new_cell.protection = copy(cell.protection)
            #    new_cell.alignment = copy(cell.alignment)
        
        # Preserve the same column width
        #width = ws.column_dimensions[cell.column_letter].width
        #new_column = new_ws.column_dimensions[new_cell.column_letter]
        #new_column.width = width
    #break


    
#Hiding gridlines
new_ws.sheet_view.showGridLines = False

# Save the new workbook to disk
new_filename = "combined.xlsx"
new_wb.save(new_filename)

# Launch the new spreadsheet
os.startfile(new_filename)

OUTPUT_2.xlsx
OUTPUT.xlsx
2
AA1:AB1
2
AC1:AP1
2
AQ1:AR1
2
AS1:BP1
2
BQ1:CN1
2
Q1:R1
2
S1:T1
2
U1:V1
2
W1:X1
2
Y1:Z1
2
A1:D2
2
E1:E2
2
F1:G1
2
H1:I1
2
K1:P1
